# Pruebas Trabajo con MLPs

In [1]:
from keras.optimizers import SGD
from tensorflow import keras
import cv2
import numpy as np
import os
from sklearn.cluster import MiniBatchKMeans
from keras import Sequential
from sklearn.base import BaseEstimator, TransformerMixin
from keras.layers import Dense, Input, Dropout, Flatten
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve
import pandas as pd
from sklearn.preprocessing import StandardScaler

2024-12-13 00:49:13.057663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734047353.122397   12223 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734047353.141675   12223 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 00:49:13.280785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def show_results_gs(gs, sort=False):
    # Extraer los resultados del GridSearchCV
    results = gs.cv_results_

    # Crear un DataFrame con los parámetros y los resultados medios
    df_results = pd.DataFrame({
        'mean_test_score': results['mean_test_score'],
        'params': results['params']
    })

    # Ordenar los resultados por el resultado promedio (opcional)
    if sort:
        df_results = df_results.sort_values(by='mean_test_score', ascending=False)

    # Mostrar los resultados
    print(df_results)

In [39]:
model = Sequential()
model.add(Input(shape=(150,)))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer = SGD(), loss='categorical_crossentropy',
                metrics=['accuracy'])

In [30]:
def leer_imagenes_rgb(dataset_path):
    jpgFiles = [fJPG for fJPG in os.listdir(dataset_path) if fJPG.endswith('.jpg')]
    bufferImages = []
    y = []

    # Extraer 50 patches aleatorios de cada imagen
    for filename in jpgFiles:
        img_path = os.path.join(dataset_path, filename)
        label = filename.split('.')[0][-2:]
        y.append(label)
        img = cv2.imread(img_path)
        # Convertir imagen de BGR a RGB
        img_color = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        bufferImages.append(img_color)

    return bufferImages, y

In [28]:
class CustomBoVWPreprocessorSamePatches(BaseEstimator, TransformerMixin):
    def __init__(self, patchSize=10, nClusters = 81, max_patches=50, norm = True):
        self.patchSize = patchSize
        self.nClusters = nClusters
        self.max_patches = max_patches
        self.norm = norm

    def fit(self, X, y=None):
        return self     
    
    def extract_centric_patches(self, img, patch_size, num_patches, seed=42):
        assert img.shape[0] == 150 and img.shape[1] == 150, "La imagen debe tener resolución 150x150."
        assert patch_size > 0, "El tamaño del patch debe ser mayor que 0."
        assert patch_size * num_patches <= 150 * 150, "Demasiados patches para esta resolución."
        
        # Calcular la cuadrícula de celdas no superpuestas
        grid_rows = 150 // patch_size
        grid_cols = 150 // patch_size
        total_cells = grid_rows * grid_cols

        assert num_patches <= total_cells, "Demasiados patches para el tamaño del patch y la resolución de la imagen."
        
        # Generar las posiciones de la cuadrícula
        all_positions = [(i * patch_size, j * patch_size) for i in range(grid_rows) for j in range(grid_cols)]

        # Calcular las distancias de las celdas al centro de la imagen
        center = (150 // 2, 150 // 2)
        distances = [
            np.sqrt((row + patch_size // 2 - center[0])**2 + (col + patch_size // 2 - center[1])**2)
            for row, col in all_positions
        ]

        # Ordenar las posiciones por cercanía al centro
        sorted_positions = [pos for _, pos in sorted(zip(distances, all_positions))]
        
        # Seleccionar los patches más céntricos
        np.random.seed(seed)
        selected_positions = sorted_positions[:num_patches]
        
        # Extraer los patches en las posiciones seleccionadas
        patches = [img[row:row + patch_size, col:col + patch_size] for row, col in selected_positions]

        #self.visualize_disjoint_patches(img, patches, selected_positions, patch_size)
        
        return np.array(patches)


    def bovw(self, images, patchSize, nClusters, maxPatches):
        bufferData = []
        for i in images:
            resized_img = cv2.resize(i, (150, 150), interpolation=cv2.INTER_AREA)
            patches = self.extract_centric_patches(resized_img, patchSize, num_patches=maxPatches, seed=27) # shape = (maxPatches,5,5,3)
            patches = np.reshape(patches, (len(patches), -1)) # shape = (maxPatches,75)
            bufferData.append(patches)
        dataIm = np.concatenate(bufferData, axis=0) # shape = (200*maxPatches,75)  n patches x 200 imágenes
        dataIm = dataIm.astype(float)
        dataIm -= np.mean(dataIm, axis=0)
        dataIm /= np.std(dataIm, axis=0)
        kmeans = MiniBatchKMeans(n_clusters=nClusters, random_state=27, verbose=False)
        kmeans.partial_fit(dataIm)

        X = np.reshape(kmeans.labels_ , (len(images),maxPatches)) # Indice de cluster de cada patch. shape = (200,maxPatches)
        resul = [np.bincount(row, minlength=nClusters) for row in X]
        resul = np.array(resul)

        if self.norm:
            contador_palabras = np.zeros((nClusters,))
            for p in range(nClusters):
                for i in resul[:,p]:
                    if i > 0:
                        contador_palabras[p] += 1
            for p in range(nClusters):
                for imagen in resul:
                    imagen[p] = imagen[p] * np.log(len(images)/(contador_palabras[p]+0.001)) # Actualizamos el valor por el logaritmo del cociente entre N y el número de imágenes en las que aparece esa palabra

        return resul

    def transform(self, images):
        # Aplica tu función de preprocesamiento aquí
        # Por ejemplo:
        # return your_custom_function(X, param1=self.param1)
        return self.bovw(images, self.patchSize, self.nClusters, self.max_patches)  # Devuelve los datos preprocesados

In [ ]:
dataset_path = '/home/pablo/Desktop/tercero/mdp/trabajo/HANDS'

imagenes, y = leer_imagenes_rgb(dataset_path)
cp = CustomBoVWPreprocessorSamePatches(patchSize=5, nClusters=150, max_patches=100)
resul = cp.transform(imagenes)

In [41]:
enc = LabelEncoder()
y_cat = enc.fit_transform(y)
y_one_hot = to_categorical(y_cat, 4)
x_train, x_test, y_train, y_test = train_test_split(resul, y_one_hot, test_size=0.2)

history = model.fit(x_train, y_train, epochs=200, batch_size=16)

Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4411 - loss: 1.3811 
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5079 - loss: 1.1524
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4561 - loss: 1.2629
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4435 - loss: 1.1683
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4023 - loss: 1.2264
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4884 - loss: 1.1600
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4695 - loss: 1.1130
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6011 - loss: 1.0345
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5230 - loss: 1.0538
Epoch 10/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4976 - loss: 1.1038
Epoch 11/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4750 - loss: 1.1170
Epoch 12/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy

In [42]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.3500


## Pruebas con BoVW

In [3]:
def leer_imagenes_rgb(dataset_path):
    jpgFiles = [fJPG for fJPG in os.listdir(dataset_path) if fJPG.endswith('.jpg')]
    bufferImages = []
    y = []

    # Extraer 50 patches aleatorios de cada imagen
    for filename in jpgFiles:
        img_path = os.path.join(dataset_path, filename)
        label = filename.split('.')[0][-2:]
        y.append(label)
        img = cv2.imread(img_path)
        # Convertir imagen de BGR a RGB
        img_color = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        bufferImages.append(img_color)

    return bufferImages, y

In [4]:
def leer_imagenes_gray(dataset_path):
    jpgFiles = [fJPG for fJPG in os.listdir(dataset_path) if fJPG.endswith('.jpg')]
    bufferImages = []
    y = []

    # Extraer 50 patches aleatorios de cada imagen
    for filename in jpgFiles:
        img_path = os.path.join(dataset_path, filename)
        label = filename.split('.')[0][-2:]
        y.append(label)
        img = cv2.imread(img_path)
        # Convertir imagen de BGR a RGB
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        bufferImages.append(img_gray)

    return bufferImages, y

In [5]:
def show_results_gs(gs, sort=False):
    # Extraer los resultados del GridSearchCV
    results = gs.cv_results_

    # Crear un DataFrame con los parámetros y los resultados medios
    df_results = pd.DataFrame({
        'mean_test_score': results['mean_test_score'],
        'params': results['params']
    })

    # Ordenar los resultados por el resultado promedio (opcional)
    if sort:
        df_results = df_results.sort_values(by='mean_test_score', ascending=False)

    # Mostrar los resultados
    return df_results

In [6]:
class CustomBoVWPreprocessorSamePatches(BaseEstimator, TransformerMixin):
    def __init__(self, patchSize=10, nClusters = 81, max_patches=50, norm = True):
        self.patchSize = patchSize
        self.nClusters = nClusters
        self.max_patches = max_patches
        self.norm = norm

    def fit(self, X, y=None):
        return self     
    
    def extract_centric_patches(self, img, patch_size, num_patches, seed=42):
        assert img.shape[0] == 150 and img.shape[1] == 150, "La imagen debe tener resolución 150x150."
        assert patch_size > 0, "El tamaño del patch debe ser mayor que 0."
        assert patch_size * num_patches <= 150 * 150, "Demasiados patches para esta resolución."
        
        # Calcular la cuadrícula de celdas no superpuestas
        grid_rows = 150 // patch_size
        grid_cols = 150 // patch_size
        total_cells = grid_rows * grid_cols

        assert num_patches <= total_cells, "Demasiados patches para el tamaño del patch y la resolución de la imagen."
        
        # Generar las posiciones de la cuadrícula
        all_positions = [(i * patch_size, j * patch_size) for i in range(grid_rows) for j in range(grid_cols)]

        # Calcular las distancias de las celdas al centro de la imagen
        center = (150 // 2, 150 // 2)
        distances = [
            np.sqrt((row + patch_size // 2 - center[0])**2 + (col + patch_size // 2 - center[1])**2)
            for row, col in all_positions
        ]

        # Ordenar las posiciones por cercanía al centro
        sorted_positions = [pos for _, pos in sorted(zip(distances, all_positions))]
        
        # Seleccionar los patches más céntricos
        np.random.seed(seed)
        selected_positions = sorted_positions[:num_patches]
        
        # Extraer los patches en las posiciones seleccionadas
        patches = [img[row:row + patch_size, col:col + patch_size] for row, col in selected_positions]

        #self.visualize_disjoint_patches(img, patches, selected_positions, patch_size)
        
        return np.array(patches)


    def bovw(self, images, patchSize, nClusters, maxPatches):
        bufferData = []
        for i in images:
            resized_img = cv2.resize(i, (150, 150), interpolation=cv2.INTER_AREA)
            patches = self.extract_centric_patches(resized_img, patchSize, num_patches=maxPatches, seed=27) # shape = (maxPatches,5,5,3)
            patches = np.reshape(patches, (len(patches), -1)) # shape = (maxPatches,75)
            bufferData.append(patches)
        dataIm = np.concatenate(bufferData, axis=0) # shape = (200*maxPatches,75)  n patches x 200 imágenes
        dataIm = dataIm.astype(float)
        dataIm -= np.mean(dataIm, axis=0)
        dataIm /= np.std(dataIm, axis=0)
        kmeans = MiniBatchKMeans(n_clusters=nClusters, random_state=27, verbose=False)
        kmeans.partial_fit(dataIm)

        X = np.reshape(kmeans.labels_ , (len(images),maxPatches)) # Indice de cluster de cada patch. shape = (200,maxPatches)
        resul = [np.bincount(row, minlength=nClusters) for row in X]
        resul = np.array(resul)

        if self.norm:
            contador_palabras = np.zeros((nClusters,))
            for p in range(nClusters):
                for i in resul[:,p]:
                    if i > 0:
                        contador_palabras[p] += 1
            for p in range(nClusters):
                for imagen in resul:
                    imagen[p] = imagen[p] * np.log(len(images)/(contador_palabras[p]+0.001)) # Actualizamos el valor por el logaritmo del cociente entre N y el número de imágenes en las que aparece esa palabra

        return resul

    def transform(self, images):
        # Aplica tu función de preprocesamiento aquí
        # Por ejemplo:
        # return your_custom_function(X, param1=self.param1)
        return self.bovw(images, self.patchSize, self.nClusters, self.max_patches)  # Devuelve los datos preprocesados

In [47]:
dataset_path = '/home/pablo/Desktop/tercero/mdp/trabajo/HANDS'

X, y = leer_imagenes_rgb(dataset_path)

In [48]:
enc = LabelEncoder()
y_cod = enc.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(X, y_cod, test_size=0.2)

In [49]:
svm = SVC()
pca = PCA()
proc = CustomBoVWPreprocessorSamePatches()
scaler = StandardScaler()

pipe = Pipeline([('proc', proc), ('pca', pca), ('scaler', scaler), ('svm', svm)])
G = {'svm__C': [0.1, 1], 'svm__kernel':['rbf', 'linear', 'sigmoid'], 'svm__gamma': [0.1, 1], 'pca__n_components': [5, 10, 20],
     'proc__patchSize': [5, 10], 'proc__max_patches': [50, 100], 'proc__nClusters': [40, 80, 200], 'proc__norm': [True, False]}
gs = GridSearchCV(pipe, G, cv=4, refit=True, verbose=10)

In [50]:
gs.fit(x_train, y_train)

Fitting 4 folds for each of 864 candidates, totalling 3456 fits
[CV 1/4; 1/864] START pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 1/4; 1/864] END pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.250 total time=   0.5s
[CV 2/4; 1/864] START pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 2/4; 1/864] END pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.250 total time=   0.5s
[CV 3/4; 1/864] START pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 3/4; 1/864] END pca__n_components=5, proc__ma

/home/pablo/Desktop/tercero/mdp/trabajo/venv/lib/python3.12/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('proc',
                                        CustomBoVWPreprocessorSamePatches()),
                                       ('pca', PCA()),
                                       ('scaler', StandardScaler()),
                                       ('svm', SVC())]),
             param_grid={'pca__n_components': [5, 10, 20],
                         'proc__max_patches': [50, 100],
                         'proc__nClusters': [40, 80, 200],
                         'proc__norm': [True, False],
                         'proc__patchSize': [5, 10], 'svm__C': [0.1, 1],
                         'svm__gamma': [0.1, 1],
                         'svm__kernel': ['rbf', 'linear', 'sigmoid']},
             verbose=10)

In [51]:
print('Los resultados en train son:')
show_results_gs(gs)

Los resultados en train son:


,mean_test_score,params
0,0.25000,"{'pca__n_components': 5, 'proc__max_patches': ..."
1,0.23750,"{'pca__n_components': 5, 'proc__max_patches': ..."
2,0.25000,"{'pca__n_components': 5, 'proc__max_patches': ..."
3,0.25000,"{'pca__n_components': 5, 'proc__max_patches': ..."
4,0.23750,"{'pca__n_components': 5, 'proc__max_patches': ..."
...,...,...
859,0.21875,"{'pca__n_components': 20, 'proc__max_patches':..."
860,0.21250,"{'pca__n_components': 20, 'proc__max_patches':..."
861,0.25625,"{'pca__n_components': 20, 'proc__max_patches':..."
862,0.21875,"{'pca__n_components': 20, 'proc__max_patches':..."


In [52]:
print('Los mejores parámetros son:')
print(gs.best_params_)
print('Con score:')
print(gs.best_score_)
print('Los resultados en test:')
y_preds = gs.predict(x_test)
print(classification_report(y_test, y_preds))
print('Score en test:')
print(gs.score(x_test, y_test))

Los mejores parámetros son:
{'pca__n_components': 10, 'proc__max_patches': 100, 'proc__nClusters': 40, 'proc__norm': False, 'proc__patchSize': 10, 'svm__C': 0.1, 'svm__gamma': 0.1, 'svm__kernel': 'linear'}
Con score:
0.31875
Los resultados en test:
              precision    recall  f1-score   support

           0       0.28      0.50      0.36        10
           1       0.00      0.00      0.00        13
           2       0.33      0.18      0.24        11
           3       0.19      0.50      0.27         6

    accuracy                           0.25        40
   macro avg       0.20      0.30      0.22        40
weighted avg       0.19      0.25      0.19        40

Score en test:
0.25


/home/pablo/Desktop/tercero/mdp/trabajo/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/pablo/Desktop/tercero/mdp/trabajo/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/pablo/Desktop/tercero/mdp/trabajo/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [53]:
dataset_path = '/home/pablo/Desktop/tercero/mdp/trabajo/HANDS'

X, y = leer_imagenes_gray(dataset_path)

enc = LabelEncoder()
y_cod = enc.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(X, y_cod, test_size=0.2)

In [54]:
svm = SVC()
pca = PCA()
proc = CustomBoVWPreprocessorSamePatches()
scaler = StandardScaler()

pipe = Pipeline([('proc', proc), ('pca', pca), ('scaler', scaler), ('svm', svm)])
G = {'svm__C': [0.1, 1], 'svm__kernel':['rbf', 'linear', 'sigmoid'], 'svm__gamma': [0.1, 1], 'pca__n_components': [5, 10, 20],
     'proc__patchSize': [5, 10], 'proc__max_patches': [50, 100], 'proc__nClusters': [40, 80, 200], 'proc__norm': [True, False]}
gs = GridSearchCV(pipe, G, cv=4, refit=True, verbose=10)

In [55]:
gs.fit(x_train, y_train)

Fitting 4 folds for each of 864 candidates, totalling 3456 fits
[CV 1/4; 1/864] START pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 1/4; 1/864] END pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.250 total time=   0.4s
[CV 2/4; 1/864] START pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 2/4; 1/864] END pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.250 total time=   0.4s
[CV 3/4; 1/864] START pca__n_components=5, proc__max_patches=50, proc__nClusters=40, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 3/4; 1/864] END pca__n_components=5, proc__ma

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('proc',
                                        CustomBoVWPreprocessorSamePatches()),
                                       ('pca', PCA()),
                                       ('scaler', StandardScaler()),
                                       ('svm', SVC())]),
             param_grid={'pca__n_components': [5, 10, 20],
                         'proc__max_patches': [50, 100],
                         'proc__nClusters': [40, 80, 200],
                         'proc__norm': [True, False],
                         'proc__patchSize': [5, 10], 'svm__C': [0.1, 1],
                         'svm__gamma': [0.1, 1],
                         'svm__kernel': ['rbf', 'linear', 'sigmoid']},
             verbose=10)

In [56]:
print('Los resultados en train son:')
show_results_gs(gs)

Los resultados en train son:


,mean_test_score,params
0,0.25000,"{'pca__n_components': 5, 'proc__max_patches': ..."
1,0.26250,"{'pca__n_components': 5, 'proc__max_patches': ..."
2,0.25000,"{'pca__n_components': 5, 'proc__max_patches': ..."
3,0.25000,"{'pca__n_components': 5, 'proc__max_patches': ..."
4,0.26250,"{'pca__n_components': 5, 'proc__max_patches': ..."
...,...,...
859,0.26250,"{'pca__n_components': 20, 'proc__max_patches':..."
860,0.26875,"{'pca__n_components': 20, 'proc__max_patches':..."
861,0.25000,"{'pca__n_components': 20, 'proc__max_patches':..."
862,0.26250,"{'pca__n_components': 20, 'proc__max_patches':..."


In [57]:
print('Los mejores parámetros son:')
print(gs.best_params_)
print('Con score:')
print(gs.best_score_)
print('Los resultados en test:')
y_preds = gs.predict(x_test)
print(classification_report(y_test, y_preds))
print('Score en test:')
print(gs.score(x_test, y_test))

Los mejores parámetros son:
{'pca__n_components': 10, 'proc__max_patches': 50, 'proc__nClusters': 80, 'proc__norm': True, 'proc__patchSize': 10, 'svm__C': 0.1, 'svm__gamma': 1, 'svm__kernel': 'sigmoid'}
Con score:
0.3625
Los resultados en test:
              precision    recall  f1-score   support

           0       0.33      0.10      0.15        10
           1       0.00      0.00      0.00        13
           2       0.20      0.18      0.19        11
           3       0.15      0.67      0.25         6

    accuracy                           0.17        40
   macro avg       0.17      0.24      0.15        40
weighted avg       0.16      0.17      0.13        40

Score en test:
0.175


## Sin PCA

In [7]:
dataset_path = '/home/pablo/Desktop/tercero/mdp/trabajo/HANDS'

X, y = leer_imagenes_rgb(dataset_path)

enc = LabelEncoder()
y_cod = enc.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(X, y_cod, test_size=0.2)

In [ ]:
svm = SVC()
proc = CustomBoVWPreprocessorSamePatches()

pipe = Pipeline([('proc', proc), ('svm', svm)])
G = {'svm__C': [0.1, 1], 'svm__kernel':['rbf', 'linear', 'sigmoid'], 'svm__gamma': [0.1, 1],
     'proc__patchSize': [5, 10], 'proc__max_patches': [50, 100], 'proc__nClusters': [80, 200], 'proc__norm': [True, False]}
gs = GridSearchCV(pipe, G, cv=4, refit=True, verbose=10)

In [11]:
gs.fit(x_train, y_train)

Fitting 4 folds for each of 192 candidates, totalling 768 fits
[CV 1/4; 1/192] START proc__max_patches=50, proc__nClusters=80, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 1/4; 1/192] END proc__max_patches=50, proc__nClusters=80, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.250 total time=   0.7s
[CV 2/4; 1/192] START proc__max_patches=50, proc__nClusters=80, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 2/4; 1/192] END proc__max_patches=50, proc__nClusters=80, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.275 total time=   0.5s
[CV 3/4; 1/192] START proc__max_patches=50, proc__nClusters=80, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf
[CV 3/4; 1/192] END proc__max_patches=50, proc__nClusters=80, proc__norm=True, proc__patchSize=5, svm__C=0.1, svm__gamma=0.1, svm__kernel=rbf;, score=0.225

/home/pablo/Desktop/tercero/mdp/trabajo/venv/lib/python3.12/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('proc',
                                        CustomBoVWPreprocessorSamePatches()),
                                       ('svm', SVC())]),
             param_grid={'proc__max_patches': [50, 100],
                         'proc__nClusters': [80, 200],
                         'proc__norm': [True, False],
                         'proc__patchSize': [5, 10], 'svm__C': [0.1, 1],
                         'svm__gamma': [0.1, 1],
                         'svm__kernel': ['rbf', 'linear', 'sigmoid']},
             verbose=10)

In [13]:
print('Los mejores parámetros son:')
print(gs.best_params_)
print('Con score:')
print(gs.best_score_)
print('Los resultados en test:')
y_preds = gs.predict(x_test)
print(classification_report(y_test, y_preds))
print('Score en test:')
print(gs.score(x_test, y_test))

Los mejores parámetros son:
{'proc__max_patches': 50, 'proc__nClusters': 200, 'proc__norm': False, 'proc__patchSize': 5, 'svm__C': 1, 'svm__gamma': 1, 'svm__kernel': 'sigmoid'}
Con score:
0.325
Los resultados en test:
              precision    recall  f1-score   support

           0       0.46      0.67      0.55         9
           1       0.33      0.40      0.36        10
           2       0.00      0.00      0.00         9
           3       0.67      0.17      0.27        12

    accuracy                           0.30        40
   macro avg       0.37      0.31      0.29        40
weighted avg       0.39      0.30      0.29        40

Score en test:
0.3


In [14]:
svm = SVC()
proc = CustomBoVWPreprocessorSamePatches()
scaler = StandardScaler()

pipe = Pipeline([('proc', proc), ('scaler', scaler), ('svm', svm)])
G = {'svm__C': [2, 4], 'svm__kernel':['sigmoid'], 'svm__gamma': [2, 4],
     'proc__patchSize': [5], 'proc__max_patches': [100], 'proc__nClusters': [150, 200], 'proc__norm': [False, True]}
gs = GridSearchCV(pipe, G, cv=4, refit=True, verbose=10)

In [15]:
gs.fit(x_train, y_train)

Fitting 4 folds for each of 16 candidates, totalling 64 fits
[CV 1/4; 1/16] START proc__max_patches=100, proc__nClusters=150, proc__norm=False, proc__patchSize=5, svm__C=2, svm__gamma=2, svm__kernel=sigmoid
[CV 1/4; 1/16] END proc__max_patches=100, proc__nClusters=150, proc__norm=False, proc__patchSize=5, svm__C=2, svm__gamma=2, svm__kernel=sigmoid;, score=0.300 total time=   0.8s
[CV 2/4; 1/16] START proc__max_patches=100, proc__nClusters=150, proc__norm=False, proc__patchSize=5, svm__C=2, svm__gamma=2, svm__kernel=sigmoid
[CV 2/4; 1/16] END proc__max_patches=100, proc__nClusters=150, proc__norm=False, proc__patchSize=5, svm__C=2, svm__gamma=2, svm__kernel=sigmoid;, score=0.375 total time=   0.5s
[CV 3/4; 1/16] START proc__max_patches=100, proc__nClusters=150, proc__norm=False, proc__patchSize=5, svm__C=2, svm__gamma=2, svm__kernel=sigmoid
[CV 3/4; 1/16] END proc__max_patches=100, proc__nClusters=150, proc__norm=False, proc__patchSize=5, svm__C=2, svm__gamma=2, svm__kernel=sigmoid;, s

GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('proc',
                                        CustomBoVWPreprocessorSamePatches()),
                                       ('scaler', StandardScaler()),
                                       ('svm', SVC())]),
             param_grid={'proc__max_patches': [100],
                         'proc__nClusters': [150, 200],
                         'proc__norm': [False, True], 'proc__patchSize': [5],
                         'svm__C': [2, 4], 'svm__gamma': [2, 4],
                         'svm__kernel': ['sigmoid']},
             verbose=10)

In [17]:
print('Los mejores parámetros son:')
print(gs.best_params_)
print('Con score:')
print(gs.best_score_)
print('Los resultados en test:')
y_preds = gs.predict(x_test)
print(classification_report(y_test, y_preds))
print('Score en test:')
print(gs.score(x_test, y_test))

Los mejores parámetros son:
{'proc__max_patches': 100, 'proc__nClusters': 150, 'proc__norm': False, 'proc__patchSize': 5, 'svm__C': 2, 'svm__gamma': 2, 'svm__kernel': 'sigmoid'}
Con score:
0.3125
Los resultados en test:
              precision    recall  f1-score   support

           0       0.22      0.40      0.29        10
           1       0.33      0.23      0.27        13
           2       0.00      0.00      0.00        11
           3       0.14      0.17      0.15         6

    accuracy                           0.20        40
   macro avg       0.17      0.20      0.18        40
weighted avg       0.19      0.20      0.18        40

Score en test:
0.2


## KNN

In [ ]:
dataset_path = '/home/pablo/Desktop/tercero/mdp/trabajo/HANDS'

X, y = leer_imagenes_rgb(dataset_path)

enc = LabelEncoder()
y_cod = enc.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(X, y_cod, test_size=0.2)

In [ ]:
svm = SVC()
proc = CustomBoVWPreprocessorSamePatches()

pipe = Pipeline([('proc', proc), ('svm', svm)])
G = {'svm__C': [0.1, 1], 'svm__kernel':['rbf', 'linear', 'sigmoid'], 'svm__gamma': [0.1, 1],
     'proc__patchSize': [5, 10], 'proc__max_patches': [50, 100], 'proc__nClusters': [80, 200], 'proc__norm': [True, False]}
gs = GridSearchCV(pipe, G, cv=4, refit=True, verbose=10)